# IBM Data Science - Applied Data Science Capstone

## Segmenting and Clustering Neighborhoods in Toronto

In [1]:
#importing necessary libraries

import numpy as np
import pandas as pd
import sklearn.cluster as sc 
!pip install folium --quiet
import folium
import json
import requests
!pip install geocoder --quiet
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Importing Completed')

Importing Completed


### Download dataset containing of Postal Code, Boroughs and Neighborhoods


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)[0]
print(df.shape)
df.head()

(180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Ignoring the cells with a Borough that is not assigned. Droping row where Borough is "Not assigned"


In [3]:
df=df[df['Borough']!='Not assigned']
print(df.shape)
df.head()

(103, 3)


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### If a cell has a borough but a not assigned Neighborhood, then the Neighborhood will be the same as the Borough

In [4]:
df.loc[df['Neighborhood']=='Not assigned','Neighborhood']=df['Borough']
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Rows will be same Postal Code will be combined into a row with comma separated Neighborhoods


In [5]:
df=df.groupby(['Postal Code','Borough'], sort=False).agg( ', '.join)
df.reset_index(inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Print the number of rows in dataframe

In [6]:
df.shape

(103, 3)

### Download dataset containing of Postal Code, Latitude and Longitude

In [7]:
!wget -q -O 'lat_long.csv'  http://cocl.us/Geospatial_data
df_ll=pd.read_csv('lat_long.csv')
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging dataset containing coordinates with that containing Borough and Neighborhoods

In [8]:
df=df.merge(df_ll,on='Postal Code')
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
df.shape

(103, 5)

### Getting the Latitude and Longitude values of Toronto City

In [10]:
address = 'Toronto'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
toronto_lat = location.latitude
toronto_lng = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(toronto_lat, toronto_lng))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


### Getting the Latitude and Longitude values of Boroughs

In [11]:
borough=df.groupby('Borough').mean().reset_index()['Borough'].values
borough

array(['Central Toronto', 'Downtown Toronto', 'East Toronto', 'East York',
       'Etobicoke', 'Mississauga', 'North York', 'Scarborough',
       'West Toronto', 'York'], dtype=object)

In [12]:
latitude=[]
longitude=[]

for num, address in zip(np.arange(len(borough)),borough):
    geolocator = Nominatim(user_agent="to_explorer")
    address=address+', ON, Canada'
    location = geolocator.geocode(address)
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude[num], longitude[num]))

The geograpical coordinate of Central Toronto, ON, Canada are 43.6534817, -79.3839347.
The geograpical coordinate of Downtown Toronto, ON, Canada are 43.6563221, -79.3809161.
The geograpical coordinate of East Toronto, ON, Canada are 43.72178945, -79.37402706301704.
The geograpical coordinate of East York, ON, Canada are 43.699971000000005, -79.33251996261595.
The geograpical coordinate of Etobicoke, ON, Canada are 43.6435559, -79.5656326.
The geograpical coordinate of Mississauga, ON, Canada are 43.590338, -79.645729.
The geograpical coordinate of North York, ON, Canada are 43.7543263, -79.44911696639593.
The geograpical coordinate of Scarborough, ON, Canada are 43.773077, -79.257774.
The geograpical coordinate of West Toronto, ON, Canada are 43.6534817, -79.3839347.
The geograpical coordinate of York, ON, Canada are 44.0007518, -79.4372217.


### Creating dataframe consisting of Boroughs and their co-ordinates

In [13]:
df_borough=pd.DataFrame({'Borough':borough,'Latitude':latitude,'Longitude':longitude})

In [14]:
df_borough

,Borough,Latitude,Longitude
0,Central Toronto,43.653482,-79.383935
1,Downtown Toronto,43.656322,-79.380916
2,East Toronto,43.721789,-79.374027
3,East York,43.699971,-79.332520
4,Etobicoke,43.643556,-79.565633
5,Mississauga,43.590338,-79.645729
6,North York,43.754326,-79.449117
7,Scarborough,43.773077,-79.257774
8,West Toronto,43.653482,-79.383935
9,York,44.000752,-79.437222


### *Map of Toronto City with Boroughs superimposed on top*

In [15]:
borough_map = folium.Map([toronto_lat,toronto_lng],zoom_start=9)

for blat, blng, bor in zip(df_borough['Latitude'], df_borough['Longitude'], df_borough['Borough']):
    folium.CircleMarker(
        location=[blat,blng],
        radius=5,
        popup=bor,
        color='#C70039',
        fill=True,
        fill_color='#C70039'
    ).add_to(borough_map)

borough_map

### *Map of Toronto City with neighborhoods superimposed on top*

In [16]:
neigh_map = folium.Map([toronto_lat-.1,toronto_lng],zoom_start=10)

for lat, lng, neigh in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    folium.CircleMarker(
        location=[lat,lng],
        radius=4,
        popup=neigh,
        color='#8417B9',
        fill=True,
        fill_color='#8417B9',
    ).add_to(neigh_map)

neigh_map

In [17]:
# The code was removed by Watson Studio for sharing.

### Getting the top 100 Venues that are in Neighborhoods within a radius of 500 meters.

In [18]:
LIMIT = 100
RADIUS = 500
venues_list=[]

for lat, lng, name in zip(df['Latitude'],df['Longitude'],df['Neighborhood']):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            RADIUS, 
            LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])    
    

### Creating dataframe consisting of Neighborhoods and Venues and their co-ordinates

In [19]:
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude', 
                         'Venue','Venue Latitude','Venue Longitude','Venue Category']
nearby_venues.shape

(2132, 7)

### *Map of Toronto City with Venues superimposed on top*

In [20]:
neigh_map = folium.Map([toronto_lat,toronto_lng],zoom_start=10)

for vlat, vlng, v in zip(nearby_venues['Venue Latitude'], nearby_venues['Venue Longitude'],nearby_venues['Venue']):
    folium.CircleMarker(
        location=[vlat,vlng],
        radius=4,
        popup=v,
        color='#3498DB',
        fill=True,
        fill_color='#3498DB',
    ).add_to(neigh_map)

neigh_map

In [21]:
nearby_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,56,56,56,56,56,56
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


### Unique categories that can be curated from all the returned Venues

In [22]:
print('Uniques categories: {} '.format(len(nearby_venues['Venue Category'].unique())))

Uniques categories: 280 


### Creating dataframe consisting of Neighborhood and Venue category

In [23]:
onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")
onehot.drop(['Neighborhood'],axis=1,inplace=True)
onehot.insert(0, 'Neighborhood', nearby_venues['Neighborhood'])
onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
neigh_cat = onehot.groupby('Neighborhood').mean().reset_index()
neigh_cat.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
neigh_cat.shape

(95, 280)

### Creating dataframe consisting of Neighborhood and Top 10 Venues

In [26]:
top_venues = 10

indicators = ['st', 'nd', 'rd']
col = ['Neighborhood']
for ind in np.arange(top_venues):
    try:
        col.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        col.append('{}th Most Common Venue'.format(ind+1))

col

['Neighborhood',
 '1st Most Common Venue',
 '2nd Most Common Venue',
 '3rd Most Common Venue',
 '4th Most Common Venue',
 '5th Most Common Venue',
 '6th Most Common Venue',
 '7th Most Common Venue',
 '8th Most Common Venue',
 '9th Most Common Venue',
 '10th Most Common Venue']

In [27]:
neigh_top_venues = pd.DataFrame(columns=col)
neigh_top_venues['Neighborhood'] = neigh_cat['Neighborhood']

print(neigh_top_venues.shape)
neigh_top_venues.head()

(95, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Alderwood, Long Branch",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Bathurst Manor, Wilson Heights, Downsview North",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bayview Village,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Bedford Park, Lawrence Manor East",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
for ind in np.arange(neigh_cat.shape[0]):
    row_cat = neigh_cat.iloc[ind, 1:]
    row_cat_sort = row_cat.sort_values(ascending=False)
    neigh_top_venues.iloc[ind, 1:] = row_cat_sort.index.values[0:top_venues]

neigh_top_venues.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Falafel Restaurant,Event Space,Farmers Market,Ethiopian Restaurant,Discount Store
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Athletics & Sports,Dance Studio,Coffee Shop,Pub,Sandwich Place,Gym,Airport Terminal,Diner
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Grocery Store,Mobile Phone Shop,Bridal Shop,Shopping Mall,Sandwich Place,Diner,Supermarket,Sushi Restaurant
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Comfort Food Restaurant,Hobby Shop,Juice Bar,Butcher,Japanese Restaurant,Café


## Cluster Neighborhoods using KMeans

In [29]:
kms = neigh_cat.drop('Neighborhood',axis=1)
print(kms.shape)
kms.head()

(95, 279)


,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
clusters = 10

kmeans = sc.KMeans(n_clusters=clusters, random_state=0).fit(kms)
kmeans.labels_[0:10] 

array([2, 0, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [31]:
neigh_top_venues.insert(0,'Cluster Label',kmeans.labels_)
neigh_top_venues.head()

,Cluster Label,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Falafel Restaurant,Event Space,Farmers Market,Ethiopian Restaurant,Discount Store
1,0,"Alderwood, Long Branch",Pizza Place,Pharmacy,Athletics & Sports,Dance Studio,Coffee Shop,Pub,Sandwich Place,Gym,Airport Terminal,Diner
2,2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Grocery Store,Mobile Phone Shop,Bridal Shop,Shopping Mall,Sandwich Place,Diner,Supermarket,Sushi Restaurant
3,2,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,2,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Comfort Food Restaurant,Hobby Shop,Juice Bar,Butcher,Japanese Restaurant,Café


In [32]:
df_tor=df.drop('Postal Code',axis=1)
df_tor.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Merging dataset containing coordinates with that containing Top 10 Categories with Borough and Neighborhoods

In [33]:
df_tor=df_tor.merge(neigh_top_venues,on='Neighborhood')
print(df_tor.shape)
df_tor.head()

(99, 15)


,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,6,Food & Drink Shop,Fireworks Store,Park,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,North York,Victoria Village,43.725882,-79.315572,2,Portuguese Restaurant,Grocery Store,Coffee Shop,Hockey Arena,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Restaurant,Historic Site,Spa
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,Furniture / Home Store,Clothing Store,Accessories Store,Coffee Shop,Event Space,Miscellaneous Shop,Gift Shop,Boutique,Vietnamese Restaurant,Cosmetics Shop
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Yoga Studio,Arts & Crafts Store,Smoothie Shop,Burrito Place,Sandwich Place,Italian Restaurant,Beer Bar,Café


### *Map of Toronto City with  Neighborhoods and Cluster superimposed on top*

In [34]:
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [35]:
cluster_map = folium.Map([toronto_lat,toronto_lng],zoom_start=10)

markers_colors = []
for clat, clon, cneigh, cluster in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Neighborhood'], df_tor['Cluster Label']):
    label = folium.Popup(cneigh+' Cluster '+str(cluster), parse_html=True)
    folium.CircleMarker(
        location=[clat, clon],
        radius=5,
        popup=cneigh,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster_map)
       
cluster_map

### Cluster 1

In [36]:
df_tor.loc[df_tor['Cluster Label'] == 0, df_tor.columns[[1] + list(range(5, df_tor.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Parkview Hill, Woodbine Gardens",Pizza Place,Pharmacy,Gastropub,Café,Intersection,Bank,Athletics & Sports,Fast Food Restaurant,Gym / Fitness Center,Pet Store
10,Glencairn,Pizza Place,Pub,Asian Restaurant,Sushi Restaurant,Japanese Restaurant,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant
15,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Pet Store,Beer Store,Pharmacy,Pizza Place,Coffee Shop,Café,Shopping Plaza,Liquor Store,German Restaurant,Deli / Bodega
50,Humber Summit,Pizza Place,Furniture / Home Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant,Drugstore
60,"Runnymede, The Junction North",Pizza Place,Caribbean Restaurant,Convenience Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
67,Westmount,Pizza Place,Sandwich Place,Intersection,Discount Store,Middle Eastern Restaurant,Coffee Shop,Chinese Restaurant,Doner Restaurant,Distribution Center,Dog Run
69,"Willowdale, Willowdale West",Pharmacy,Grocery Store,Pizza Place,Bank,Butcher,Coffee Shop,Donut Shop,Discount Store,Distribution Center,Dog Run
74,"Kingsview Village, St. Phillips, Martin Grove ...",Mobile Phone Shop,Sandwich Place,Pizza Place,Bus Line,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
79,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Pharmacy,Italian Restaurant,Noodle House,Rental Car Location,Thai Restaurant,Bank,Gas Station
86,"South Steeles, Silverstone, Humbergate, Jamest...",Grocery Store,Pizza Place,Pharmacy,Fried Chicken Joint,Beer Store,Sandwich Place,Fast Food Restaurant,Construction & Landscaping,Comic Shop,Falafel Restaurant


### Cluster 2

In [37]:
df_tor.loc[df_tor['Cluster Label'] == 1, df_tor.columns[[1] + list(range(5, df_tor.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,"York Mills, Silver Hills",Cafeteria,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Diner


### Cluster 3

In [38]:
df_tor.loc[df_tor['Cluster Label'] == 2, df_tor.columns[[1] + list(range(5, df_tor.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,Portuguese Restaurant,Grocery Store,Coffee Shop,Hockey Arena,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,"Regent Park, Harbourfront",Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Restaurant,Historic Site,Spa
3,"Lawrence Manor, Lawrence Heights",Furniture / Home Store,Clothing Store,Accessories Store,Coffee Shop,Event Space,Miscellaneous Shop,Gift Shop,Boutique,Vietnamese Restaurant,Cosmetics Shop
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Yoga Studio,Arts & Crafts Store,Smoothie Shop,Burrito Place,Sandwich Place,Italian Restaurant,Beer Bar,Café
6,Don Mills,Gym,Asian Restaurant,Sporting Goods Shop,Beer Store,Japanese Restaurant,Restaurant,Coffee Shop,Supermarket,Chinese Restaurant,Baseball Field
7,Don Mills,Gym,Asian Restaurant,Sporting Goods Shop,Beer Store,Japanese Restaurant,Restaurant,Coffee Shop,Supermarket,Chinese Restaurant,Baseball Field
9,"Garden District, Ryerson",Clothing Store,Coffee Shop,Bubble Tea Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Bakery,Ramen Restaurant
12,Woodbine Heights,Park,Spa,Pharmacy,Skating Rink,Video Store,Beer Store,Athletics & Sports,Curling Ice,Drugstore,Distribution Center
13,St. James Town,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Italian Restaurant,Restaurant,Clothing Store,Moroccan Restaurant,Cosmetics Shop
14,Humewood-Cedarvale,Field,Trail,Tennis Court,Hockey Arena,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore


### Cluster 4

In [39]:
df_tor.loc[df_tor['Cluster Label'] == 3, df_tor.columns[[1] + list(range(5, df_tor.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
55,"Humberlea, Emery",Food Service,Baseball Field,Filipino Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio
97,"Old Mill South, King's Mill Park, Sunnylea, Hu...",Baseball Field,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner


### Cluster 5

In [40]:
df_tor.loc[df_tor['Cluster Label'] == 4, df_tor.columns[[1] + list(range(5, df_tor.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,"Malvern, Rouge",Fast Food Restaurant,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


### Cluster 6

In [41]:
df_tor.loc[df_tor['Cluster Label'] == 5, df_tor.columns[[1] + list(range(5, df_tor.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"Rouge Hill, Port Union, Highland Creek",Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
30,Scarborough Village,Playground,Construction & Landscaping,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant


### Cluster 7

In [42]:
df_tor.loc[df_tor['Cluster Label'] == 6, df_tor.columns[[1] + list(range(5, df_tor.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Food & Drink Shop,Fireworks Store,Park,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
19,Caledonia-Fairbanks,Park,Women's Store,Pool,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
33,"East Toronto, Broadview North (Old East York)",Park,Intersection,Convenience Store,Metro Station,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
58,Lawrence Park,Park,Bus Line,Swim School,Yoga Studio,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
61,Weston,Park,Convenience Store,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
63,York Mills West,Park,Convenience Store,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
80,"Moore Park, Summerhill East",Gym,Park,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
94,"The Kingsway, Montgomery Road, Old Mill North",Park,River,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


### Cluster 8

In [43]:
df_tor.loc[df_tor['Cluster Label'] == 7, df_tor.columns[[1] + list(range(5, df_tor.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Woburn,Coffee Shop,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant


### Cluster 9

In [44]:
df_tor.loc[df_tor['Cluster Label'] == 8, df_tor.columns[[1] + list(range(5, df_tor.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Diner,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio


### Cluster 10

In [45]:
df_tor.loc[df_tor['Cluster Label'] == 9,df_tor.columns[[1] + list(range(5, df_tor.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
82,"Milliken, Agincourt North, Steeles East, L'Amo...",Park,Playground,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
88,Rosedale,Park,Playground,Trail,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
